In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-08 16:42:41.588585: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 16:42:41.588652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 16:42:41.592163: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 16:42:41.613127: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 16:42:44.266824: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../BreaKHis_v1"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/7909
[INFO]: Processed 2000/7909
[INFO]: Processed 3000/7909
[INFO]: Processed 4000/7909
[INFO]: Processed 5000/7909
[INFO]: Processed 6000/7909
[INFO]: Processed 7000/7909
(7909, 224, 224, 3)
(7909,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-08 16:48:01.298722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


248/248 [==============================] - 483s 2s/step
(7909, 100352)


In [9]:
data = features_flatten

In [11]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [11]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8963337547408344
Sensitivity: 0.9318600368324125
Specificity: 0.8185483870967742
AUC-ROC: 0.8752042119645934
MCC: 0.7572548013149937
Precision: 0.9183303085299456
F1 Score: 0.9250457038391224

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8988621997471555
Sensitivity: 0.9410681399631676
Specificity: 0.8064516129032258
AUC-ROC: 0.8737598764331966
MCC: 0.7617255656822657
Precision: 0.9141323792486583
F1 Score: 0.9274047186932849

[INFO] Fold 3 / 10 for LR
Accuracy: 0.9013906447534766
Sensitivity: 0.9373848987108656
Specificity: 0.8225806451612904
AUC-ROC: 0.8799827719360779
MCC: 0.7687207662034725
Precision: 0.9204339963833634
F1 Score: 0.9288321167883212

[INFO] Fold 4 / 10 for LR
Accuracy: 0.9152970922882427
Sensitivity: 0.9465930018416207
Specificity: 0.8467741935483871
AUC-ROC: 0.8966835976950039
MCC: 0.8015516360940248
Precision: 0.9311594202898551
F1 Score: 0.938812785388128

[INFO] Fold 5 / 10 for LR
Accuracy: 0.9102402022756005
Sensitivity: 0.9539594843462247
Specificity: 0.8145161290322581
AUC-ROC: 0.8842378066892413
MCC: 0.7881399915108255
Precision: 0.9184397163120568
F1 Score: 0.935862691960253

[INFO] Fold 6 / 10 for LR
Accuracy: 0.911504424778761
Sensitivity: 0.9539594843462247
Specificity: 0.818

In [12]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.7610619469026548
Sensitivity: 0.8305709023941068
Specificity: 0.6088709677419355
AUC-ROC: 0.7197209350680212
MCC: 0.4419099430913657
Precision: 0.822992700729927
F1 Score: 0.8267644362969753

[INFO] Fold 2 / 10 for NB
Accuracy: 0.7648546144121365
Sensitivity: 0.8158379373848987
Specificity: 0.6532258064516129
AUC-ROC: 0.7345318719182559
MCC: 0.46235881191573014
Precision: 0.8374291115311909
F1 Score: 0.8264925373134328

[INFO] Fold 3 / 10 for NB
Accuracy: 0.7724399494310998
Sensitivity: 0.8158379373848987
Specificity: 0.6774193548387096
AUC-ROC: 0.7466286461118044
MCC: 0.4834827996451692
Precision: 0.847036328871893
F1 Score: 0.8311444652908068

[INFO] Fold 4 / 10 for NB
Accuracy: 0.7623261694058154
Sensitivity: 0.8139963167587477
Specificity: 0.6491935483870968
AUC-ROC: 0.7315949325729222
MCC: 0.4565688961346875
Precision: 0.8355387523629489
F1 Score: 0.8246268656716419

[INFO] Fold 5 / 10 for NB
Accuracy: 0.7850821744627055
Sensitivity: 0.854511

In [13]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.7825537294563844
Sensitivity: 0.9576427255985267
Specificity: 0.39919354838709675
AUC-ROC: 0.6784181369928117
MCC: 0.4583547927651274
Precision: 0.7772795216741405
F1 Score: 0.858085808580858

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.7749683944374209
Sensitivity: 0.9650092081031307
Specificity: 0.3588709677419355
AUC-ROC: 0.6619400879225331
MCC: 0.43761065934147125
Precision: 0.767203513909224
F1 Score: 0.8548123980424143

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.7812895069532237
Sensitivity: 0.9373848987108656
Specificity: 0.43951612903225806
AUC-ROC: 0.6884505138715619
MCC: 0.4543573472186593
Precision: 0.7854938271604939
F1 Score: 0.8547439126784215

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.7951959544879899
Sensitivity: 0.9558011049723757
Specificity: 0.4435483870967742
AUC-ROC: 0.6996747460345749
MCC: 0.4939054377473136
Precision: 0.7899543378995434
F1 Score: 0.865

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.7825537294563844
Sensitivity: 0.955801104972

In [14]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.809102402022756
Sensitivity: 0.9373848987108656
Specificity: 0.5282258064516129
AUC-ROC: 0.7328053525812392
MCC: 0.5316423553492361
Precision: 0.8130990415335463
F1 Score: 0.8708297690333618

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.8268015170670038
Sensitivity: 0.9447513812154696
Specificity: 0.5685483870967742
AUC-ROC: 0.7566498841561219
MCC: 0.5784994141775801
Precision: 0.8274193548387097
F1 Score: 0.8822012037833189

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.8305941845764855
Sensitivity: 0.9465930018416207
Specificity: 0.5766129032258065
AUC-ROC: 0.7616029525337136
MCC: 0.5884219218954274
Precision: 0.8303715670436187
F1 Score: 0.8846815834767642

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.8293299620733249
Sensitivity: 0.9484346224677717
Specificity: 0.5685483870967742
AUC-ROC: 0.7584915047822731
MCC: 0.585144976540676
Precision: 0.8279742765273312
F1 Score: 0.8841201716738198

[INFO] Fold 5 / 10 for Random For

In [12]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4886, number of negative: 2232
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 24.349356 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15370145
[LightGBM] [Info] Number of data points in the train set: 7118, number of used features: 98697
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.686429 -> initscore=0.783476
[LightGBM] [Info] Start training from score 0.783476
Accuracy: 0.854614412136536
Sensitivity: 0.9410681399631676
Specificity: 0.6653225806451613
AUC-ROC: 0.8031953603041644
MCC: 0.6505069882489477
Precision: 0.8602693602693603
F1 Score: 0.8988566402814423

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4886, number of negative: 2232
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 24.993347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

In [13]:
# Save results to a CSV file
results_df.to_csv('DS3_ResNet50_LGBM.csv', index=False)

In [14]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LGBM,0.871918,0.94548,0.710887,0.828183,0.69406,0.877597,0.910215,4114.910156,9727.391654
